<a href="https://colab.research.google.com/github/Prashast7/Spark/blob/main/Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark_Series


In [1]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySparkPractice").getOrCreate()

print("SparkSession created successfully!")

SparkSession created successfully!


In [3]:
# Create a sample DataFrame
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
columns = ["Name", "ID"]
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

+-------+---+
|   Name| ID|
+-------+---+
|  Alice|  1|
|    Bob|  2|
|Charlie|  3|
+-------+---+



In [5]:
netflix_df = spark.read.csv("netflix_titles.csv", header = True, inferSchema = False)

In [6]:
netflix_df.printSchema()
netflix_df.show(2)


root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+-------------------

In [7]:
schema = """show_id string,
type string,
title string,
 director string,
 cast string,
 country string,
 date_added string,
 release_year integer,
 rating string,
 duration string,
 listed_in string,
description string """

In [12]:
netflix_df = spark.read.csv("netflix_titles.csv", header = True, schema = schema)

In [9]:
from pyspark.sql.functions import *

In [10]:
null_counts = {column: netflix_df.filter(col(column).isNull()) for column in netflix_df.columns}

In [22]:
print(null_counts)

{'show_id': DataFrame[show_id: string, type: string, title: string, director: string, cast: string, country: string, date_added: string, release_year: int, rating: string, duration: string, listed_in: string, description: string], 'type': DataFrame[show_id: string, type: string, title: string, director: string, cast: string, country: string, date_added: string, release_year: int, rating: string, duration: string, listed_in: string, description: string], 'title': DataFrame[show_id: string, type: string, title: string, director: string, cast: string, country: string, date_added: string, release_year: int, rating: string, duration: string, listed_in: string, description: string], 'director': DataFrame[show_id: string, type: string, title: string, director: string, cast: string, country: string, date_added: string, release_year: int, rating: string, duration: string, listed_in: string, description: string], 'cast': DataFrame[show_id: string, type: string, title: string, director: string, c

In [11]:
netflix_df.show(4)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         NULL|Septem

In [13]:
netflix_df = netflix_df.withColumn("date_added_clean", to_date(netflix_df["date_added"], "MMMM d, yyyy"))

In [14]:
netflix_df.show(4)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|date_added_clean|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|      2021-09-25|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|      2021-09-24|
|     s3|T

In [15]:
netflix_df = netflix_df.withColumn("year_added", year("date_added_clean"))

In [17]:
netflix_df.groupBy("year_added").agg(count("show_id")).orderBy(desc("count(show_id)")).show()

+----------+--------------+
|year_added|count(show_id)|
+----------+--------------+
|      2019|          1997|
|      2020|          1872|
|      2018|          1623|
|      2021|          1491|
|      2017|          1162|
|      2016|           418|
|      NULL|           120|
|      2015|            72|
|      2014|            23|
|      2011|            13|
|      2013|            10|
|      2012|             3|
|      2009|             2|
|      2008|             2|
|      2010|             1|
+----------+--------------+



In [18]:
netflix_df = netflix_df.withColumn("month_added", month("date_added_clean"))

In [19]:
netflix_df.groupBy("month_added").agg(count("title")).show()

+-----------+------------+
|month_added|count(title)|
+-----------+------------+
|         12|         796|
|       NULL|         118|
|          1|         722|
|          6|         723|
|          3|         732|
|          5|         625|
|          9|         763|
|          4|         757|
|          8|         746|
|          7|         818|
|         10|         754|
|         11|         696|
|          2|         557|
+-----------+------------+



In [18]:
netflix_df.groupBy("country").agg(count(when(col("type") == "Movie", True)).alias("movie_count")).orderBy(desc("movie_count")).show(5)

+--------------+-----------+
|       country|movie_count|
+--------------+-----------+
| United States|       2047|
|         India|        893|
|          NULL|        440|
|United Kingdom|        206|
|        Canada|        122|
+--------------+-----------+
only showing top 5 rows



In [22]:
netflix_df.groupBy("country").agg(count(when(col("type")=="TV Show", True)).alias("show_count")).orderBy(desc("show_count")).show(5)

+--------------+----------+
|       country|show_count|
+--------------+----------+
| United States|       758|
|          NULL|       391|
|United Kingdom|       213|
|         Japan|       169|
|   South Korea|       158|
+--------------+----------+
only showing top 5 rows



In [29]:
netflix_df.groupBy("director").agg(count("show_id").alias("show_count")).filter("show_count > 10").orderBy(desc("show_count")).show()

+--------------------+----------+
|            director|show_count|
+--------------------+----------+
|                NULL|      2636|
|       Rajiv Chilaka|        19|
|Raúl Campos, Jan ...|        18|
|        Marcus Raboy|        16|
|         Suhas Kadav|        16|
|           Jay Karas|        14|
| Cathy Garcia-Molina|        13|
|     Youssef Chahine|        12|
|     Martin Scorsese|        12|
|         Jay Chapman|        12|
|    Steven Spielberg|        11|
+--------------------+----------+



In [27]:
netflix_df.createOrReplaceTempView("netflix")
spark.sql("""
select director,count(distinct show_id) as count_show
from netflix
group by director
having count_show > 10
order by count_show desc""").show()

+--------------------+----------+
|            director|count_show|
+--------------------+----------+
|                NULL|      2636|
|       Rajiv Chilaka|        19|
|Raúl Campos, Jan ...|        18|
|        Marcus Raboy|        16|
|         Suhas Kadav|        16|
|           Jay Karas|        14|
| Cathy Garcia-Molina|        13|
|     Youssef Chahine|        12|
|     Martin Scorsese|        12|
|         Jay Chapman|        12|
|    Steven Spielberg|        11|
+--------------------+----------+



In [36]:
netflix_df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|date_added_clean|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|      2021-09-25|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|      2021-09-24|
|     s3|T

# Netlfix Analysis

In [35]:
# prompt: Which genre appears most frequently on Netflix (based on the listed_in column)?
netflix_df.select(explode(split(col("listed_in"), ", ")).alias("genre")).groupBy("genre").count().orderBy(desc("count")).show()


+--------------------+-----+
|               genre|count|
+--------------------+-----+
|International Movies| 2748|
|              Dramas| 2419|
|            Comedies| 1670|
|International TV ...| 1350|
|       Documentaries|  866|
|  Action & Adventure|  857|
|           TV Dramas|  762|
|  Independent Movies|  751|
|Children & Family...|  641|
|     Romantic Movies|  616|
|         TV Comedies|  580|
|           Thrillers|  573|
|      Crime TV Shows|  469|
|            Kids' TV|  451|
|          Docuseries|  395|
|    Music & Musicals|  373|
|   Romantic TV Shows|  370|
|       Horror Movies|  357|
|     Stand-Up Comedy|  343|
|          Reality TV|  254|
+--------------------+-----+
only showing top 20 rows



In [40]:
netflix_df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|date_added_clean|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+----------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|      2021-09-25|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|      2021-09-24|
|     s3|T

In [43]:
netflix_df.select(explode(split(col("cast"), ", ")).alias("actor")).groupBy("actor").count().orderBy(desc("count")).show()

+-----------------+-----+
|            actor|count|
+-----------------+-----+
|      Anupam Kher|   43|
|   Shah Rukh Khan|   35|
|    Julie Tejwani|   33|
| Takahiro Sakurai|   32|
| Naseeruddin Shah|   32|
|     Rupa Bhimani|   31|
|     Akshay Kumar|   30|
|          Om Puri|   30|
|        Yuki Kaji|   29|
| Amitabh Bachchan|   28|
|     Paresh Rawal|   28|
|      Boman Irani|   27|
|     Vincent Tong|   26|
|      Rajesh Kava|   26|
|   Kareena Kapoor|   25|
|    Andrea Libman|   25|
|Samuel L. Jackson|   24|
|      John Cleese|   24|
|      Tara Strong|   23|
|  Fred Tatasciore|   23|
+-----------------+-----+
only showing top 20 rows



In [47]:
netflix_df.select("title","country","date_added_clean").orderBy(desc("date_added_clean")).show(3)

+--------------------+-------------+----------------+
|               title|      country|date_added_clean|
+--------------------+-------------+----------------+
|Dick Johnson Is Dead|United States|      2021-09-25|
|       Blood & Water| South Africa|      2021-09-24|
|           Ganglands|         NULL|      2021-09-24|
+--------------------+-------------+----------------+
only showing top 3 rows

